# Quick Start

This notebook demonstrates a basic workflow with the goes_ortho package, to download a short time series of GOES-R ABI imagery (the ABI-L1b-RadC product), and build a zarr file.

The notebook ends by creating a gif image (with the helpful [geogif](https://geogif.readthedocs.io/en/latest/) package) for us to get a quick preview of the imagery.

In [1]:
import goes_ortho as go

# we'll use geogif and xarray to look at the imagery
import geogif
import xarray as xr

 ╭─goes2go──────────────────────────────────────────────╮
 │ INFO: Created a default config file.                 │
 │ You may view/edit goes2go's configuration here:      │
 │       /home/cdalden/.config/goes2go/config.toml      │
 ╰──────────────────────────────────────────────────────╯



First, specify the time range, location bounds, satellite, product (and if applicable, band and variable) that we'd like to access.

We will also need to provide an API key for [OpenTopography.org](https://portal.opentopography.org/requestService?service=api) which you can create with a free account. This allows goes_ortho to access digital elevation models to perform the orthorectification step.

The workflow below was developed to read a json file containing information about what we'd like to download. This was done to 1) allow these functions to run through github actions (still an experimental feature) and 2) keep a record of datasets we've downloaded. This is something that may change in the near future since it adds an unnecessary step for most use cases.

In [2]:
startDatetime = "2023-06-10T00:00:00Z"
endDatetime = "2023-06-10T00:59:00Z"
[min_lon, min_lat, max_lon, max_lat] = [-109, 37, -108.5, 37.5]
satellite = "goes16"
product = "ABI-L1b-RadC"
band = 13
variable = "Rad"
OPENTOPO_API_KEY = "41d14aae7e761c0de3e8f99aa4fd24d9"

Using the data above, we make the required json file.

In [3]:
# Make request file from user input
go.get_data.make_request_json(
    f"{satellite}-build-zarr-b{band}",
    startDatetime,
    endDatetime,
    [min_lon, min_lat, max_lon, max_lat],
    satellite,
    product,
    band,
    variable,
    OPENTOPO_API_KEY,
)

'goes16-build-zarr-b13.json'

Now we can provide the json file to the `build_zarr` function. This function downloads, clips, orthorectifies, and merges all imagery into a single zarr file.

In [4]:
# Download GOES imagery and build zarr file
go.get_data.build_zarr(f"{satellite}-build-zarr-b{band}.json", 'goespy')


Files will be downloaded and then cropped to these bounds:
	(-109,37.5).	.(-108.5,37.5)



	(-109,37).	.(-108.5,37)

For each S3 bucket, download the corresponding observations
Downloading... the product ABI-L1b-RadC 
OR_ABI-L1b-RadC-M6C13_G16_s20231610001184_e20231610003569_c20231610004020.nc: [##########] 100%  4.195512 MB/4.195512 MB

In [ ]:
go.get_data.build_zarr?

Signature: go.get_data.build_zarr(downloadRequest_filepath)
Docstring: <no docstring>
File:      ~/miniforge3/envs/goesenv/lib/python3.13/site-packages/goes_ortho/get_data.py
Type:      function

To preview what our new zarr file looks like, we can open it with xarray, and use geogif to make a little gif animation.

In [ ]:
# open the zarr file
ds = xr.open_zarr(f"{satellite}-build-zarr-b{band}.zarr")
# select our variable of interest
da = ds[variable]

# create the gif animation
gif_bytes = geogif.dgif(
    da,
    fps=5,
    cmap="Greys_r",
    date_format="%Y-%m-%d %H:%M:%S",
    date_position="ul",
    bytes=True,
).compute()

# write gif to file
with open(f"{satellite}-b{band}.gif", "wb") as f:
    f.write(gif_bytes)

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/xarray/backends/plugins.py:110: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  external_backend_entrypoints = backends_dict_from_pkg(entrypoints_unique)


FileNotFoundError: Unable to find group: file:///home/cdalden/goes_work/goes18-build-zarr-b13.zarr

Take a look at the gif image we just made:

<img src="goes18-b2.gif" alt="GOES-18 animation" width="400"/>